# Realizar consultas por fecha como argumento

In [ ]:
# Se importan las librerías
# Para más información de datetime: https://docs.python.org/3/library/datetime.html

import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

: 

In [ ]:
# Se establece el argumento
arg_date = '2022-12-30'

: 

In [ ]:
# strptime: https://www.geeksforgeeks.org/python-datetime-strptime-function/
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

: 

In [ ]:
# Se establece el servicio y se accede al bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')

: 

Se obtienen todos los objetos cuya fecha sea mayor o igual que el argumento de fecha:

In [ ]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]

: 

In [ ]:
# La línea anterior paso a paso:
for obj in bucket.objects.all():    
    if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt:
        print(obj.key)
        # print(obj.key.split("/"))
        # print(obj.key.split("/")[0])
        # Con datetime.strptime se transforma a un objeto datetime
        # print(datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d'))
        # print(datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date())

: 

In [ ]:
objects

: 

In [ ]:
# Se obtiene el objeto inicial por posición en el bucket objects
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

: 

In [ ]:
df_init.columns

: 

In [ ]:
# Concatenar todos los objetos con pandas, como son más, tarda más
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = pd.concat([df,df_all], ignore_index=True)

: 

In [ ]:
# Se obtienen los primeros 10
df_all.head(10)

: 

In [ ]:
# Filtrar y extraer sólo algunas columnas
columns = ["ISIN", "Mnemonic", "Date", "Time", "StartPrice", "EndPrice", "MinPrice", "MaxPrice", "TradedVolume"]
df_all = df_all.loc[:, columns]

: 

In [ ]:
df_all

: 

In [ ]:
# Eliminar missing values: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# inplace=True elimina los NaN y modifica el df original, si setea como false, creará un nuevo df sin los NaN eliminados.
df_all.dropna(inplace=True)

: 

In [ ]:
# Consultar la forma del df para verificar si se borró alguna fila
df_all.shape

: 

## Obtener precio de apertura por ISIN y día

In [ ]:
# Se creará una nueva columna en el df con el precio de apertura.
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

: 

In [ ]:
# Código anterior paso a paso
df_sorted = df_all.sort_values(by=['Time'])
print("df_sorted\n", df_sorted)
df_grouped = df_sorted.groupby(['ISIN', 'Date'])
print("df_grouped\n", df_grouped)
df_start = df_grouped['StartPrice']
print("df_start\n", df_start)
df_transformed = df_start.transform('first')
print("df_transformed\n", df_transformed)

: 

In [ ]:
# Consultar por clave ISIN
df_all[df_all['ISIN']=='AT0000A0E9W5']

: 

## Obtener el precio de cierre por ISIN y día

In [ ]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')

: 

In [ ]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

: 

## Aggregations

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html

In [ ]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

: 

In [ ]:
df_all

: 

### Porcentaje Cambio Anterior Cierre

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html

df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

: 

In [ ]:
df_all

: 

In [ ]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

: 

In [ ]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

: 

In [ ]:
df_all = df_all.round(decimals=2)

: 

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
df_all.reset_index(inplace=True)

: 

Obtener el df final que cumple con un criterio de fecha

In [ ]:
df_all = df_all[df_all.Date >= arg_date]

: 

In [ ]:
df_all

: 

## Salvar a S3 en formato .parquet

Apache Parquet es un formato de almacenamiento en columnas disponible para cualquier proyecto en el ecosistema de Hadoop, independientemente de la elección del framework de procesamiento de datos, el modelo de datos o el lenguaje de programación.

https://parquet.apache.org/docs/overview/

In [ ]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

: 

In [ ]:
'''
BytesIO() se utiliza para guardar los datos como bytes en un búfer en memoria. 
Con este bloque de código se está almacenando el archivo .parquet en el bucket target. Es decir, se guarda la 
información directamente en la nube de amazon y no en el disco duro local.
'''

out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('xetra-ajlj')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

: 

In [ ]:
s3.Object(bucket_name='xetra-vcl-itspr', key='xetra_daily_report_20220218_134804.parquet')

: 

## Leer el archivo subido a s3

Esto se realiza para verificar que el reporte fue guardado correctamente.

In [ ]:
for obj in bucket_target.objects.all():
    print(obj.key)

: 

In [ ]:
# Se lee el archivo parquet
prq_obj = bucket_target.Object(key='xetra_daily_report_20230217_232455.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

: 

In [ ]:
df_report

: 